In [97]:
# LIB
import os
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import joblib
from sklearn.metrics import accuracy_score, confusion_matrix
DATA_DIR = os.sep.join(['bbc_news_Data','bbc-fulltext (document classification)','bbc'])
CATEGORY = 'category'
DOCUMENT_ID = 'document_id'
#TEXT = 'text'
TITLE = 'title' # remove this element for the demo
#STORY = 'story'
INPUT = 'input'
LABEL = 'label'

In [98]:
# READ Data from text
frame = defaultdict(list)
for dir_name, _, file_names in os.walk(DATA_DIR):
    try:
        file_names.remove('README.TXT')
        file_names.remove('.DS_Store')
    except:
        pass
    for file_name in file_names:
        frame[LABEL].append(os.path.basename(dir_name))
        name = os.path.splitext(file_name)[0]
        frame[DOCUMENT_ID].append(name)
        path = os.path.join(dir_name, file_name)
        with open(path,'r', encoding='unicode_escape') as file:
            frame[INPUT].append(file.read())
df = pd.DataFrame.from_dict(frame)

# DATA STATUS NOW
df.head()


,label,document_id,input
0,business,001,Ad sales boost Time Warner profit\n\nQuarterly...
1,business,002,Dollar gains on Greenspan speech\n\nThe dollar...
2,business,003,Yukos unit buyer faces loan claim\n\nThe owner...
3,business,004,High fuel prices hit BA's profits\n\nBritish A...
4,business,005,Pernod takeover talk lifts Domecq\n\nShares in...


In [99]:
#VERTFY_DATA

In [100]:
# DATA Encoding

# LABEL
df[LABEL].unique()
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df[LABEL].values)
y


# TEXT
nltk.download('stopwords')
stopword_e = stopwords.words('english')
def clean_text(text):
    # decontraction: https://stackoverflow.com/a/47091490/7445772
    # specific
    text =re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", 'can not', text)

    #general
    text = re.sub(r"n\'t", 'not', text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"\'s", "is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)

    # remove line breaks  \r \n \t remove from string
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\t', ' ')
    text = text.replace('\\n', ' ')

    # remove stop words
    text = ' '.join(word for word in text.split() if word not in stopword_e)

    # remove special words
    text = re.sub('[^A-Za-z0-9]+', " ", text)
    text = text.lower()
    return text

input_processed = []
for txt in tqdm(df[INPUT].values):
    txt_p = clean_text(txt)
    input_processed.append(txt_p)

#Vect.
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
X = text_transformer.fit_transform(input_processed)

X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2)
X_train
y_train

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\23742\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|████████████████████████████████████████████████████████████████████████████| 2225/2225 [00:01<00:00, 1584.90it/s]


array([2, 3, 4, ..., 2, 0, 2])

In [101]:
# Model trtain
model = linear_model.LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)
joblib.dump(model,'LR_model')
#model = joblib.load('LR_model')
lr_prediction = model.predict(X_test)
print('accurancy: ',accuracy_score(lr_prediction,y_test))



accurancy:  0.9730337078651685


In [102]:
#Verify txt test
vframe = defaultdict(list)
with open('varify.txt','r', encoding='unicode_escape')as file:
    vframe['txt'].append(file.read())
vdf = pd.DataFrame.from_dict(vframe)
vdf['txt'] = vdf['txt'].apply(lambda x: clean_text(x))
x_v = text_transformer.fit_transform(vdf['txt'])
model_l = joblib.load('LR_model')
res = model_l.predict(x_v)


ValueError: X has 233 features per sample; expecting 746340